<a href="https://colab.research.google.com/github/Amanuel94/kaggle/blob/main/ResNet_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf

In [15]:
class ResidualUnit(tf.keras.layers.Layer):
  def __init__(self, filters, strides = 1, activation = "relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = tf.keras.activations.get(activation)
    self.main_layers = [
        tf.keras.layers.Conv2D(filters, 3, strides = strides, padding = "same", use_bias = False),
        self.activation,
        tf.keras.layers.Conv2D(filters, 3, strides = 1, padding = "same", use_bias = False),
    ]
    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
          tf.keras.layers.Conv2D(filters, 1, strides = strides, padding = "same", use_bias =False),
      ]

  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)

    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z  = layer(skip_Z)

    return self.activation(skip_Z + Z)

In [16]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, 7,
                                 strides = 2,
                                 input_shape=[224, 224, 3],
                                 padding = "same",
          use_bias = False))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size = 3,
                                    strides = 2,
                                    padding = "same"))
prev_filters = 64
maps = [64]*3 + [128]*4 + [256]*6 + [512]*3
for filters in maps:
  strides = 1
  if filters != prev_filters:
    strides = 2

  model.add(ResidualUnit(filters, strides = strides))
  prev_filters = filters

model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation = "softmax"))

model.summary()



Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_42 (Conv2D)                   │ (None, 112, 112, 64)        │           9,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_41               │ (None, 112, 112, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_6 (Activation)            │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_17 (ResidualUnit)      │ (None, 56, 56, 64)          │          73,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_18 (ResidualUnit)      │ (None, 56, 56, 64)          │          73,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_19 (ResidualUnit)      │ (None, 56, 56, 64)          │          73,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_20 (ResidualUnit)      │ (None, 28, 28, 128)         │         229,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_21 (ResidualUnit)      │ (None, 28, 28, 128)         │         294,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_22 (ResidualUnit)      │ (None, 28, 28, 128)         │         294,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_23 (ResidualUnit)      │ (None, 28, 28, 128)         │         294,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_24 (ResidualUnit)      │ (None, 14, 14, 256)         │         917,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_25 (ResidualUnit)      │ (None, 14, 14, 256)         │       1,179,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_26 (ResidualUnit)      │ (None, 14, 14, 256)         │       1,179,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_27 (ResidualUnit)      │ (None, 14, 14, 256)         │       1,179,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_28 (ResidualUnit)      │ (None, 14, 14, 256)         │       1,179,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_29 (ResidualUnit)      │ (None, 14, 14, 256)         │       1,179,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_30 (ResidualUnit)      │ (None, 7, 7, 512)           │       3,670,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_31 (ResidualUnit)      │ (None, 7, 7, 512)           │       4,718,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ residual_unit_32 (ResidualUnit)      │ (None, 7, 7, 512)           │       4,718,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 512)                 │              

 Total params: 21,273,034 (81.15 MB)

 Trainable params: 21,272,906 (81.15 MB)

 Non-trainable params: 128 (512.00 B)